# Instalación Spark en Google Colab

Primero hemos de instalar Java como prerrequisito

In [7]:
!apt-get install openjdk-8-jdk-headless -qq

'apt-get' is not recognized as an internal or external command,
operable program or batch file.


El siguiente paso es descargar y configurar Spark:

In [2]:
!wget -q https://downloads.apache.org/spark/spark-3.5.0/spark-3.5.0-bin-hadoop3.tgz

In [3]:
!tar xvf spark-3.5.0-bin-hadoop3.tgz

tar: Error opening archive: Failed to open 'spark-3.5.0-bin-hadoop3.tgz'


Instalamos pyspark para trabajar mediante Python

In [4]:
# instalar pyspark
!pip install -q pyspark

Configuramos las variables de entorno

In [5]:
# Variables de Entorno
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/spark-3.5.0-bin-hadoop3"

Cargamos las librerías

In [6]:
# Cargar Pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("s8a").master("local[*]").getOrCreate()
spark

FileNotFoundError: [WinError 2] The system cannot find the file specified

Y ya podemos empezar a desarrollar mediante Python en Spark

In [7]:
# Ejemplo RDD
# Suma de los 100 primeros números
sc = spark.sparkContext
rdd = sc.parallelize(range(100 + 1))
rdd.sum()

5050

In [8]:
# Ejemplo DataFrame
!wget -q https://gist.githubusercontent.com/jsdario/6d6c69398cb0c73111e49f1218960f79/raw/8d4fc4548d437e2a7203a5aeeace5477f598827d/el_quijote.txt

quijoteTxt = spark.read.text("el_quijote.txt")
print(quijoteTxt.count())  # número de filas del DataFrame - 2186
print(quijoteTxt.first())  # primera fila - Row(value='DON QUIJOTE DE LA MANCHA')
# Transformamos un DataFrame en otro nuevo
lineasConQuijote = quijoteTxt.filter(quijoteTxt.value.contains("Quijote"))  # DataFrame con las líneas que contiene la palabra Quijote
lineasConQuijote.show()
print(lineasConQuijote.count())  # cantidad de líneas con la palabra Quijote - 584
# Las transformaciones se pueden encadenar
quijoteTxt.filter(quijoteTxt.value.contains("Quijote")).count()     # idem - 584


2186
Row(value='DON QUIJOTE DE LA MANCHA')
+--------------------+
|               value|
+--------------------+
|CAPÍTULO 1: Que ...|
|Imaginábase el p...|
|Limpias, pues, su...|
|Capítulo 2: Que ...|
|Estos pensamiento...|
|Fuese llegando a ...|
|El lenguaje no en...|
|Pensó el huéspe...|
|porque era la mej...|
|Nunca fuera cabal...|
|O Rocinante, que ...|
|Preguntáronle si...|
|Estando en esto, ...|
|Capítulo 3: Dond...|
|Y así, fatigado ...|
|Preguntóle si tr...|
|quedaban sanos de...|
|Antojósele en es...|
|No le parecieron ...|
|Hechas, pues, de ...|
+--------------------+
only showing top 20 rows

584


584

In [9]:
# Ejemplo datos Collab
archivo = './sample_data/california_housing_train.csv'
dfSpark = spark.read.csv(archivo, inferSchema=True, header=True)

# imprimir tipo de archivo
dfSpark.printSchema()

root
 |-- longitude: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- housing_median_age: double (nullable = true)
 |-- total_rooms: double (nullable = true)
 |-- total_bedrooms: double (nullable = true)
 |-- population: double (nullable = true)
 |-- households: double (nullable = true)
 |-- median_income: double (nullable = true)
 |-- median_house_value: double (nullable = true)



In [10]:
dfSpark.describe().show()

+-------+-------------------+------------------+------------------+-----------------+-----------------+------------------+-----------------+------------------+------------------+
|summary|          longitude|          latitude|housing_median_age|      total_rooms|   total_bedrooms|        population|       households|     median_income|median_house_value|
+-------+-------------------+------------------+------------------+-----------------+-----------------+------------------+-----------------+------------------+------------------+
|  count|              17000|             17000|             17000|            17000|            17000|             17000|            17000|             17000|             17000|
|   mean|-119.56210823529375|  35.6252247058827| 28.58935294117647|2643.664411764706|539.4108235294118|1429.5739411764705|501.2219411764706| 3.883578100000021|207300.91235294117|
| stddev| 2.0051664084260357|2.1373397946570867|12.586936981660406|2179.947071452777|421.4994515798648| 1